In [1]:
!pip install tensorflow_graphics

     |████████████████████████████████| 348kB 9.2MB/s 
     |████████████████████████████████| 471kB 16.9MB/s 
     |████████████████████████████████| 634kB 30.4MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for OpenEXR: filename=OpenEXR-1.3.2-cp36-cp36m-linux_x86_64.whl size=188438 sha256=c5af8386242a04a46e0ddc1d0a58924dd2ce10ebceb6d4ba834ef075e771e700
  Stored in directory: /root/.cache/pip/wheels/41/06/9f/c7dc838815b0e7dfc7d7dc19cc3d677edb47594d8489adc62a
  Created wheel for psutil: filename=psutil-5.7.3-cp36-cp36m-linux_x86_64.whl size=281535 sha256=1bb94ccbe5283d77ebbc09bda2fa39975f31a60997ff4c0b7a090867d8f30ec0
  Stored in directory: /root/.cache/pip/wheels/42/32/da/8b12fd6b138c733efd03cfde6c6c8191a32842f9e82aa45fbf
Successfully built OpenEXR psutil
  Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Success

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import glob
import os
%tensorflow_version 1.x%
import tensorflow as tf

from tensorflow_graphics.nn.layer import graph_convolution as graph_conv
from tensorflow_graphics.notebooks import mesh_segmentation_dataio as dataio
from tensorflow_graphics.notebooks import mesh_viewer

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x%`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
import numpy as np
from tensorflow_graphics.notebooks import threejs_visualization

SEGMENTATION_COLORMAP = np.array(
    ((165, 242, 12), (89, 12, 89), (165, 89, 165), (242, 242, 165),
     (242, 165, 12), (89, 12, 12), (165, 12, 12), (165, 89, 242), (12, 12, 165),
     (165, 12, 89), (12, 89, 89), (165, 165, 89), (89, 242, 12), (12, 89, 165),
     (242, 242, 89), (165, 165, 165)),
    dtype=np.float32) / 255.0


class Viewer(object):
  """A ThreeJS based viewer class for viewing 3D meshes."""

  def _mesh_from_data(self, data):
    """Creates a dictionary of ThreeJS mesh objects from numpy data."""
    if 'vertices' not in data or 'faces' not in data:
      raise ValueError('Mesh Data must contain vertices and faces')
    vertices = np.asarray(data['vertices'])
    faces = np.asarray(data['faces'])
    material = self.context.THREE.MeshLambertMaterial.new_object({
        'color': 0xfffacd,
        'vertexColors': self.context.THREE.NoColors,
        'side': self.context.THREE.DoubleSide,
    })
    mesh = {'vertices': vertices, 'faces': faces}
    if 'vertex_colors' in data:
      mesh['vertex_colors'] = np.asarray(data['vertex_colors'])
      material = self.context.THREE.MeshLambertMaterial.new_object({
          'color': 0xfffacd,
          'vertexColors': self.context.THREE.VertexColors,
          'side': self.context.THREE.DoubleSide,
      })
    mesh['material'] = material
    return mesh

  def __init__(self, source_mesh_data):
    context = threejs_visualization.build_context()
    self.context = context
    light1 = context.THREE.PointLight.new_object(0x808080)
    light1.position.set(10., 10., 10.)
    light2 = context.THREE.AmbientLight.new_object(0x808080)
    lights = (light1, light2)

    camera = threejs_visualization.build_perspective_camera(
        field_of_view=30, position=(0.0, 0.0, 4.0))

    mesh = self._mesh_from_data(source_mesh_data)
    geometries = threejs_visualization.triangular_mesh_renderer([mesh],
                                                                lights=lights,
                                                                camera=camera,
                                                                width=600,
                                                                height=600)

    self.geometries = geometries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
path_to_model_zip = tf.keras.utils.get_file(
    'model.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/model.zip',
    extract=True)

local_model_dir = os.path.join(os.path.dirname(path_to_model_zip), 'model')
test_data_files = [
    os.path.join('/content/drive/My Drive/Colab Notebooks/cgProject/new/Dancer.tfrecords')
]

## Model Definition

Given a mesh with V vertices and D-dimensional per-vertex input features (e.g.
vertex position, normal), we would like to create a network capable of
classifying each vertex to a part label. Let's first create a mesh encoder that
encodes each vertex in the mesh into C-dimensional logits, where C is the number
of parts. First we use 1x1 convolutions to change input feature dimensions,
followed by a sequence of feature steered graph convolutions and ReLU
non-linearities, and finally 1x1 convolutions to logits, which are used for
computing softmax cross entropy as described below.

Note that this model does not use any form of pooling, which is outside the scope of this notebook.

![](https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/mesh_segmentation_model_def.png)

In [20]:
MODEL_PARAMS = {
    'num_filters': 8,
    'num_classes': 16,
    'encoder_filter_dims': [32, 64, 128],
}


def mesh_encoder(batch_mesh_data, num_filters, output_dim, conv_layer_dims):
  """A mesh encoder using feature steered graph convolutions.

    The shorthands used below are
      `B`: Batch size.
      `V`: The maximum number of vertices over all meshes in the batch.
      `D`: The number of dimensions of input vertex features, D=3 if vertex
        positions are used as features.

  Args:
    batch_mesh_data: A mesh_data dict with following keys
      'vertices': A [B, V, D] `float32` tensor of vertex features, possibly
        0-padded.
      'neighbors': A [B, V, V] `float32` sparse tensor of edge weights.
      'num_vertices': A [B] `int32` tensor of number of vertices per mesh.
    num_filters: The number of weight matrices to be used in feature steered
      graph conv.
    output_dim: A dimension of output per vertex features.
    conv_layer_dims: A list of dimensions used in graph convolution layers.

  Returns:
    vertex_features: A [B, V, output_dim] `float32` tensor of per vertex
      features.
  """
  batch_vertices = batch_mesh_data['vertices']

  # Linear: N x D --> N x 16.
  vertex_features = tf.keras.layers.Conv1D(16, 1, name='lin16')(batch_vertices)

  # graph convolution layers
  for dim in conv_layer_dims:
    with tf.variable_scope('conv_%d' % dim):
      vertex_features = graph_conv.feature_steered_convolution_layer(
          vertex_features,
          batch_mesh_data['neighbors'],
          batch_mesh_data['num_vertices'],
          num_weight_matrices=num_filters,
          num_output_channels=dim)
    vertex_features = tf.nn.relu(vertex_features)

  # Linear: N x 128 --> N x 256.
  vertex_features = tf.keras.layers.Conv1D(
      256, 1, name='lin256')(
          vertex_features)
  vertex_features = tf.nn.relu(vertex_features)

  # Linear: N x 256 --> N x output_dim.
  vertex_features = tf.keras.layers.Conv1D(
      output_dim, 1, name='lin_output')(
          vertex_features)

  return vertex_features

Given a mesh encoder, let's define a model_fn for a custom
[tf.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)
for vertex classification using softmax cross entropy loss. A tf.Estimator model_fn returns the ops necessary to perform training, evaluation, or predictions given inputs and a number of other parameters. Recall that the
vertex tensor may be zero-padded (see Dataset Pipeline above), hence we must mask out the contribution from the padded values.

In [21]:
def get_learning_rate(params):
  """Returns a decaying learning rate."""
  global_step = tf.train.get_or_create_global_step()
  learning_rate = tf.train.exponential_decay(
      params['init_learning_rate'],
      global_step,
      params['lr_decay_steps'],
      params['lr_decay_rate'])
  return learning_rate

def model_fn(features, labels, mode, params):
  """Returns a mesh segmentation model_fn for use with tf.Estimator."""
  logits = mesh_encoder(features, params['num_filters'], params['num_classes'],
                        params['encoder_filter_dims'])
  predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
  outputs = {
      'vertices': features['vertices'],
      'triangles': features['triangles'],
      'num_vertices': features['num_vertices'],
      'num_triangles': features['num_triangles'],
      'predictions': predictions,
  }
  # For predictions, return the outputs.
  if mode == tf.estimator.ModeKeys.PREDICT:
    outputs['labels'] = features['labels']
    return tf.estimator.EstimatorSpec(mode=mode, predictions=outputs)
  # Loss
  # Weight the losses by masking out padded vertices/labels.
  vertex_ragged_sizes = features['num_vertices']
  mask = tf.sequence_mask(vertex_ragged_sizes, tf.shape(labels)[-1])
  loss_weights = tf.cast(mask, dtype=tf.float32)
  loss = tf.losses.sparse_softmax_cross_entropy(
      logits=logits, labels=labels, weights=loss_weights)
  # For training, build the optimizer.
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(
        learning_rate=get_learning_rate(params),
        beta1=params['beta'],
        epsilon=params['adam_epsilon'])
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      train_op = optimizer.minimize(
          loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # For eval, return eval metrics.
  eval_ops = {
      'mean_loss':
          tf.metrics.mean(loss),
      'accuracy':
          tf.metrics.accuracy(
              labels=labels, predictions=predictions, weights=loss_weights)
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_ops)

## Train the model from scratch

Now let's train the mesh segmentation model from scratch. First we will download the train dataset files, and use tf.Estimator.train_and_evaluate to train a model.

Note: Training code is provided inside colab for demonstration, and may be slow. For optimal performance, consider running the training process as a command line process, and a tensorboard process to track.

In [22]:
path_to_train_data_zip = tf.keras.utils.get_file(
    'train_data.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/train_data.zip',
    extract=True)

train_data_files = glob.glob(
    os.path.join(os.path.dirname(path_to_train_data_zip), '*train*.tfrecords'))

retrain_model_dir = os.path.join(local_model_dir, 'retrain')

In [23]:

train_io_params = {
    'batch_size': 8,
    'parallel_threads': 8,
    'is_training': True,
    'shuffle': True,
    'sloppy': True,
}

eval_io_params = {
    'batch_size': 8,
    'parallel_threads': 8,
    'is_training': False,
    'shuffle': False
}


def train_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          train_data_files, train_io_params)


def eval_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          test_data_files, eval_io_params)


train_params = {
    'beta': 0.9,
    'adam_epsilon': 1e-8,
    'init_learning_rate': 0.001,
    'lr_decay_steps': 10000,
    'lr_decay_rate': 0.95,
}

train_params.update(MODEL_PARAMS)

checkpoint_delay = 120  # Checkpoint every 2 minutes.
max_steps = 2000  # Number of training steps.

config = tf.estimator.RunConfig(
    log_step_count_steps=1,
    save_checkpoints_secs=checkpoint_delay,
    keep_checkpoint_max=3)

classifier = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=retrain_model_dir,
    config=config,
    params=train_params)
train_spec = tf.estimator.TrainSpec(input_fn=train_fn, max_steps=max_steps)
eval_spec = tf.estimator.EvalSpec(
    input_fn=eval_fn,
    steps=None,
    start_delay_secs=2 * checkpoint_delay,
    throttle_secs=checkpoint_delay)

print('Start training & eval.')
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)
print('Train and eval done.')


INFO:tensorflow:Using config: {'_model_dir': '/root/.keras/datasets/model/retrain', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 120, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f0c721518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Start training & eval.
INFO:tensorflow:Not using Distribute Coordinator.
INFO

## Test model & visualize results

Now that we have defined the model, let's load the weights from the trained model downloaded above and use tf.Estimator.predict to predict the part labels for meshes in the test dataset.

In [24]:
test_io_params = {
    'is_training': False,
    'sloppy': False,
    'shuffle': True,
    'repeat': False
}
test_tfrecords = test_data_files

def predict_fn():
  return dataio.create_input_from_dataset(dataio.create_dataset_from_tfrecords,
                                          test_tfrecords,
                                          test_io_params)


test_predictions = classifier.predict(input_fn=predict_fn)


Run the following cell repeatedly to cycle through the meshes in the test sequence. The left view shows the input mesh, and the right view shows the predicted part labels.

In [25]:
prediction = next(test_predictions)
input_mesh_data = {
    'vertices': prediction['vertices'],
    'faces': prediction['triangles'],
}
predicted_mesh_data = {
    'vertices': prediction['vertices'],
    'faces': prediction['triangles'],
    'vertex_colors': mesh_viewer.SEGMENTATION_COLORMAP[prediction['predictions']],
}

#input_viewer = mesh_viewer.Viewer(input_mesh_data)
#prediction_viewer = mesh_viewer.Viewer(predicted_mesh_data)

input_viewer = Viewer(input_mesh_data)
prediction_viewer = Viewer(predicted_mesh_data)

Output hidden; open in https://colab.research.google.com to view.